In [ ]:
import grpc
import swoq_pb2
import swoq_pb2_grpc
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
player_id = '6616b1c5bd0a697480a68319'

In [ ]:
with grpc.insecure_channel('localhost:5009') as channel:
    stub = swoq_pb2_grpc.TrainingStub(channel)
    startResponse = stub.StartGame(swoq_pb2.StartRequest(playerId=player_id))

print(f'{startResponse.result=}')

game_id = startResponse.gameId
print(f'{game_id=}')
height = startResponse.height
width = startResponse.width
print(f'{height=}, {width=}')

map = np.array(startResponse.state.map).reshape(height, width)
plt.figure()
plt.imshow(map)

In [ ]:
with grpc.insecure_channel('localhost:5009') as channel:
    stub = swoq_pb2_grpc.TrainingStub(channel)
    moveResponse = stub.Move(swoq_pb2.MoveRequest(gameId=game_id, direction=swoq_pb2.NORTH))

print(f'{moveResponse.result=}')
print(f'{moveResponse.state.finished=}')

map = np.array(moveResponse.state.map).reshape(height, width)
plt.figure()
plt.imshow(map)